In [4]:
import folium
import branca.colormap as cm
import numpy as np
import pandas as pd

def generate_peakhour_map_point(csv_file, hour):
    """
    Generate a scatter map of Peak Hourly Volume(PHV) with colors varying by PHV value based on a CSV file.
    
    Parameters:
    - csv_file (str): Path to the CSV file, which must contain the columns "LATITUDE", "LONGITUDE", "HOUR" and "PHV".
    
    Returns:
    - folium.Map object
    """
    # 1. Read the CSV file
    df = pd.read_csv(csv_file)
    filtered_df = df.copy()

    # Calculate the difference in POSTMILE values
    filtered_df["POSTMILE_DIFF"] = filtered_df["PM"].diff()
    
    # Filter only rows where the hour is the given hour
    filtered_df = filtered_df[filtered_df["HOUR"] == hour]

    # Keep only rows where the difference is >= 2
    filtered_df = filtered_df[filtered_df["POSTMILE_DIFF"] >= 2]

    # 2. Create a Folium map (set the center to the mean of coordinates)
    m = folium.Map(
        location=[df["LATITUDE"].mean() + 0.5, df["LONGITUDE"].mean()], 
        zoom_start=8, 
        tiles="OpenStreetMap"
    )

    # 3. Create a color map (for the scatter plot with a gradient of PHV values)
    colormap = cm.LinearColormap(
        colors=["blue", "green", "yellow", "red"],  
        vmin=df["PHV"].min(),  
        vmax=df["PHV"].max()
    )

    # 4. add legend to the map
    colormap.caption = "PHV traffic data"
    colormap.add_to(m)

    if filtered_df.empty:
        print(f"No data for hour {hour}")
        return m

    # 5. Loop through the data and plot the scatter points
    for _, row in filtered_df.iterrows():
        lat, lon, PHV = row["LATITUDE"], row["LONGITUDE"], row["PHV"]
        color = colormap(PHV) # Calculate color based on PHV value

        folium.CircleMarker(
            location=(lat, lon),
            radius=4,  # Marker size
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=folium.Popup(f"PHV: {PHV}", parse_html=True)  # Display PHV value on hover
        ).add_to(m)

    # 6. Return the map object
    return m


In [5]:
import io
from PIL import Image
import selenium

In [6]:
for i in range(0, 24):
    PHV_map = generate_peakhour_map_point("peak_hour_with_coordinates.csv", i)
    if PHV_map is None:
        print(f"Hour {i} done")
        continue
    map_title = f"Peak Traffic Volume Hour {i}"
    title_html = f'<h1 style="position:absolute;z-index:100000;left:20vw" >{map_title}</h1>'
    PHV_map.get_root().html.add_child(folium.Element(title_html))
    img_data = PHV_map._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    img.save(f'Hour_{i}.png')
    print(f"Hour {i} done")
    # display(PHV_map)

No data for hour 0
Hour 0 done
Hour 1 done
No data for hour 2
Hour 2 done
No data for hour 3
Hour 3 done
No data for hour 4
Hour 4 done
Hour 5 done
Hour 6 done
Hour 7 done
Hour 8 done
Hour 9 done
Hour 10 done
Hour 11 done
Hour 12 done
Hour 13 done
Hour 14 done
Hour 15 done
Hour 16 done
Hour 17 done
Hour 18 done
No data for hour 19
Hour 19 done
No data for hour 20
Hour 20 done
Hour 21 done
No data for hour 22
Hour 22 done
No data for hour 23
Hour 23 done


In [7]:
import folium
import branca.colormap as cm
import numpy as np

df2 = pd.read_csv("peak_hour_with_coordinates.csv")
df = pd.read_csv("i5_2022_coordinates.csv")
df = df[df["CNTY"].isin(["SD", "LA", "ORA"])].copy()


# 1️ Create a Folium map
m = folium.Map(
    location=[df2["LATITUDE"].mean() +0.5, df2["LONGITUDE"].mean()], 
    zoom_start=8, 
    tiles="OpenStreetMap"
)

# 2️ Color mapping (used for gradient lines and legend)
colormap = cm.LinearColormap(
    colors=["blue", "green", "yellow", "red"],  # Color gradient
    vmin=df["VEHICLE_AADT_TOTAL"].min(),  
    vmax=df["VEHICLE_AADT_TOTAL"].max()
)

# 3️ Draw smooth gradient lines
num_subsegments = 10  # Split each line segment into 10 parts to make the color transition smoother
for i in range(len(df) - 1):
    lat1, lon1, aadt1 = df.iloc[i][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]
    lat2, lon2, aadt2 = df.iloc[i + 1][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]

    # Generate interpolated latitudes and longitudes (subdivide the line)
    lats = np.linspace(lat1, lat2, num_subsegments)
    lons = np.linspace(lon1, lon2, num_subsegments)
    aadts = np.linspace(aadt1, aadt2, num_subsegments)  # Interpolate AADT as well for smooth color transition

    # Draw smaller line segments with color gradients
    for j in range(num_subsegments - 1):
        color = colormap(aadts[j])  # Calculate the color for the current small segment


        folium.PolyLine(
            locations=[(lats[j], lons[j]), (lats[j + 1], lons[j + 1])],
            color=color,
            weight=5,
            opacity=0.8
        ).add_to(m)


# 4 Add AADT color legend
colormap.caption = "AADT Traffic Data"
colormap.add_to(m)


map_title = f"AADT For LA, SD, ORA County 2022"
title_html = f'<h1 style="position:absolute;z-index:100000;left:10vw" >{map_title}</h1>'
m.get_root().html.add_child(folium.Element(title_html))
img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(f'AADT County.png')

# 5 Display the map
display(m)
print(f"Done")


Done
